# Evaluations 

In [47]:
import os
import re
import numpy as np
import pandas as pd
import copy

os.environ["CMBAGENT_DEBUG"] = "false"
os.environ["ASTROPILOT_DISABLE_DISPLAY"] = "false"

import cmbagent
path_to_targets = '/Users/milind24/Evals/cmbagent/'

# Creation of tasks.csv

In [51]:
import csv
import os
import random

os.makedirs(f"{path_to_targets}/targets", exist_ok=True)

# Extended list of unique trigonometric expressions (at least 50 unique ones)
trig_functions = [
    "sin(x)", "cos(x)", "tan(x)", 
    "sin(x) + cos(x)", "sin(x) - cos(x)", "sin(x) * cos(x)", "sin(x)/cos(x)", 
    "sin(x)**2 + cos(x)**2", "cos(x)**2 - sin(x)**2", "1 / (1 + tan(x)**2)",
    "tan(x) * sin(x)", "sin(2*x)", "cos(2*x)", "tan(2*x)",
    "sin(x)/x", "cos(x)/x", "tan(x)/x",
    "sin(x + np.pi/4)", "cos(x - np.pi/3)", "tan(x + np.pi/6)",
    "np.arcsin(np.clip(x, -1, 1))", "np.arccos(np.clip(x, -1, 1))", "np.arctan(x)",
    "np.sinh(x)", "np.cosh(x)", "np.tanh(x)",
    "np.sin(x)**3", "np.cos(x)**3", "np.tan(x)**3",
    "np.sin(x) * np.tan(x)", "np.cos(x) * np.tan(x)", "np.sin(x) * np.cos(x) * np.tan(x)",
    "np.sin(x)**2 - np.cos(x)**2", "np.tan(x)**2 - np.sin(x)**2",
    "np.sin(x + x**2)", "np.cos(x**2)", "np.tan(x**2)",
    "np.sin(x)/np.sqrt(1 + x**2)", "np.cos(x)/np.sqrt(1 + x**2)", "np.tan(x)/np.sqrt(1 + x**2)",
    "np.sin(np.exp(x))", "np.cos(np.exp(x))", "np.tan(np.exp(x))",
    "np.arcsin(x / (1 + x**2))", "np.arccos(x / (1 + x**2))", "np.arctan(1 / (1 + x**2))",
    "np.sinh(2*x)", "np.cosh(2*x)", "np.tanh(2*x)","np.sinh(x**2)","np.cosh(x**2)"
]

# Ensure exactly 50 unique expressions
assert len(trig_functions) >= 50, "Need at least 50 unique trigonometric expressions"
selected_expressions = trig_functions  # No repetitions

# Prepare tasks
tasks = []
for i, expr in enumerate(selected_expressions):
    # Create a safe filename based on expression
    clean_expr = expr.replace("np.", "").replace("**", "^").replace("/", "_over_")\
                     .replace("+", "_plus_").replace("-", "_minus_")\
                     .replace("*", "_times_").replace("(", "").replace(")", "").replace(" ", "")
    
    task = {
        "task_id": f"task{i+1}",
        "input": f"""
Compute {expr} for x = np.linspace(-1,1,100).

Instructions: 
 - Save results in a csv file named result.csv with columns "x" and "{expr}"
""",
        "target_file_path": f"{path_to_targets}/targets/target_x_{clean_expr}.csv",
        "initial_agent": "engineer",
        "engineer_model": random.choice(["gpt-4o", "gpt-4o-mini", "gemini-2.5-pro-preview-03-25"])
    }
    tasks.append(task)

# Write to CSV in BigCodeBench-Hard format
output_csv = "tasks.csv"
with open(output_csv, mode="w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["task_id", "input", "target_file_path", "initial_agent", "engineer_model"])
    writer.writeheader()
    for task in tasks:
        writer.writerow(task)

print(f"✅ CSV file '{output_csv}' created successfully with 50 unique trigonometric tasks.")


✅ CSV file 'tasks.csv' created successfully with 50 unique trigonometric tasks.


# Creation of target.csvs

In [56]:
import numpy as np
import pandas as pd
import csv
import os
import re

# Load the tasks from the previously created CSV
input_csv = "tasks.csv"

def inject_np(expr):
    # Matches only bare trig or math function names, not ones already prefixed
    pattern = r'(?<![\w.])(sin|cos|tan|arcsin|arccos|arctan|sinh|cosh|tanh|exp|sqrt|pi)(?![\w.])'
    return re.sub(pattern, r'np.\1', expr)


# Read tasks
with open(input_csv, mode="r", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    tasks = list(reader)

# Iterate through tasks and compute results
for task in tasks:
    try:
        # Extract expression from the input string
        input_lines = task["input"].strip().split("\n")
        expr_line = [line for line in input_lines if line.startswith("Compute")][0]
        expr_raw = expr_line.replace("Compute", "").replace("for x = np.linspace(-1,1,100).", "").strip()
        expr = inject_np(expr_raw)

        # Prepare computation
        x = np.linspace(-1, 1, 100)
        y = eval(expr, {"np": np, "x": x})

        # Save to CSV
        df = pd.DataFrame({"x": x, expr_raw: y})
        os.makedirs(os.path.dirname(task["target_file_path"]), exist_ok=True)
        df.to_csv(task["target_file_path"], index=False)
        print(f"✅ Saved: {task['target_file_path']}")
    except Exception as e:
        print(f"❌ Failed: {task['task_id']} — {e}")


✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_sinx.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_cosx.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_tanx.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_sinx_plus_cosx.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_sinx_minus_cosx.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_sinx_times_cosx.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_sinx_over_cosx.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_sinx^2_plus_cosx^2.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_cosx^2_minus_sinx^2.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_1_over_1_plus_tanx^2.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_tanx_times_sinx.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_sin2_times_x.csv
✅ Saved: /Users/milind24/Evals/cmbagent//targets/target_x_cos2_times_x.csv
✅ Saved: /Users

# Reading the tasks

In [57]:
import csv

# Path to your CSV containing the tasks
csv_file_path = 'tasks.csv'

def load_tasks_from_csv(csv_file_path):
    tasks = []
    
    # Open CSV file and read contents
    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            task = {
                "input": row['input'],
                "metadata": {
                    "target_file_path": row['target_file_path'],
                    "initial_agent": row['initial_agent'],
                    "engineer_model": row['engineer_model'],
                }
            }
            tasks.append(task)
    
    return tasks

# Load tasks from CSV
mytasks = load_tasks_from_csv(csv_file_path)

# Optionally print out the loaded tasks
for idx, task in enumerate(mytasks, start=1):
    print(f"Task {idx}:")
    print(f"Input: {task['input']}")
    print(f"Metadata: {task['metadata']}\n")


Task 1:
Input: 
Compute sin(x) for x = np.linspace(-1,1,100).

Instructions: 
 - Save results in a csv file named result.csv with columns "x" and "sin(x)"

Metadata: {'target_file_path': '/Users/milind24/Evals/cmbagent//targets/target_x_sinx.csv', 'initial_agent': 'engineer', 'engineer_model': 'gpt-4o'}

Task 2:
Input: 
Compute cos(x) for x = np.linspace(-1,1,100).

Instructions: 
 - Save results in a csv file named result.csv with columns "x" and "cos(x)"

Metadata: {'target_file_path': '/Users/milind24/Evals/cmbagent//targets/target_x_cosx.csv', 'initial_agent': 'engineer', 'engineer_model': 'gpt-4o-mini'}

Task 3:
Input: 
Compute tan(x) for x = np.linspace(-1,1,100).

Instructions: 
 - Save results in a csv file named result.csv with columns "x" and "tan(x)"

Metadata: {'target_file_path': '/Users/milind24/Evals/cmbagent//targets/target_x_tanx.csv', 'initial_agent': 'engineer', 'engineer_model': 'gemini-2.5-pro-preview-03-25'}

Task 4:
Input: 
Compute sin(x) + cos(x) for x = np.lins

In [40]:
# task1 = {"input":r"""
# Compute sin(x) for x = np.linspace(-1,1,100).

# Instructions: 
#  - Save results in a csv file named result.csv with columns "x" and "sin(x)"
# """,
#          "metadata": {"target_file_path": f"{path_to_targets}/targets/target_x_sinx.csv",
#                       "initial_agent": "engineer",
#                       "engineer_model": 'gemini-2.5-pro-preview-03-25',
#                       }
#          }

# task2 = {"input":r"""
# Compute cos(x) for x = np.linspace(-1,1,100).

# Instructions: 
#  - Save results in a csv file named result.csv with columns "x" and "cos(x)"
# """,
#          "metadata": {"target_file_path": f"{path_to_targets}/targets/target_x_sinx.csv",
#                       "initial_agent": "engineer",
#                       "engineer_model": 'gpt-4o-mini',
#                       }
#          }


# task3 = {"input":r"""
# Compute sin(x) for x = np.linspace(-1,1,100).

# Instructions: 
#  - Save results in a csv file named result.csv with columns "x" and "sin(x)"
# """,
#          "metadata": {"target_file_path": f"{path_to_targets}/targets/target_x_sinx.csv",
#                       "initial_agent": "engineer",
#                       "engineer_model": 'gpt-4o',
#                       }
#          }

# mytasks = [task3]#, task2, task3]






In [59]:
def get_solution(cmbagent_results,metadata):
    chat_history = cmbagent_results['chat_history']
    final_context = cmbagent_results['final_context']

    
    # Load target and output dataframes
    target_file = metadata['target_file_path']
    df_target = pd.read_csv(target_file)
    
    database_full_path = os.path.join(
        final_context['work_dir'],
        final_context['database_path'],
        "result.csv"
    )
    resolved_path = os.path.abspath(database_full_path)  # resolves the '..'
    try: 
        df_output = pd.read_csv(resolved_path)
    except FileNotFoundError as e:
        return ["FAILED", str(e)]
        
        
    
    print("Agent Output:")
    print(df_output)
    print("Target Data:")
    print(df_target)
    
    target_columns = list(df_target.columns)
    output_columns = list(df_output.columns)
    
    # Check if columns perfectly match (including order)
    if target_columns != output_columns:
        missing = [col for col in target_columns if col not in output_columns]
        extra = [col for col in output_columns if col not in target_columns]
        
        error_parts = []
        if missing:
            error_parts.append(f"Missing columns: {missing}")
        if extra:
            error_parts.append(f"Extra columns: {extra}")
        error_message = "Column mismatch: " + "; ".join(error_parts)
        return ["FAILED", str(error_message)]
        # return 
    
    # Check that numerical values in each column match within 1% tolerance
    try:
        for col in target_columns:
            target_values = df_target[col].to_numpy()
            output_values = df_output[col].to_numpy()
            
            # For numeric columns, assert that values are within 1% of each other
            if pd.api.types.is_numeric_dtype(df_target[col]):
                assert np.allclose(target_values, output_values, rtol=0.01, atol=0), \
                    f"Values in column '{col}' do not match within 1% tolerance."
            else:
                # For non-numeric columns, require exact match
                assert (target_values == output_values).all(), \
                    f"Values in column '{col}' do not match exactly."
    except AssertionError as e:
        return ["FAILED", str(e)]
    
    return "PASSED"

In [60]:
def my_agent(task, metadata):

    results = cmbagent.one_shot(task,
                                max_rounds=10,
                                initial_agent=metadata['initial_agent'],
                                engineer_model=metadata['engineer_model']
                                )


    return get_solution(results,metadata)

In [61]:
results = []

for task in mytasks:
    print(task['input'])
    print(task['metadata'])
    result = my_agent(task['input'],task['metadata'])
    # result = my_agent(input, tasks['metadata'])
    results.append(result)


Compute sin(x) for x = np.linspace(-1,1,100).

Instructions: 
 - Save results in a csv file named result.csv with columns "x" and "sin(x)"

{'target_file_path': '/Users/milind24/Evals/cmbagent//targets/target_x_sinx.csv', 'initial_agent': 'engineer', 'engineer_model': 'gpt-4o'}
Message from _User:


Compute sin(x) for x = np.linspace(-1,1,100).

Instructions: 
 - Save results in a csv file named result.csv with columns "x" and "sin(x)"


Calling engineer...



Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
gpt-4o-2024-08-06,engineer,0.00687,1952,199,2151


Message from engineer:




Forwarding content...



Calling engineer_response_formatter...



...


Calling terminator...



Model,agent,Cost,Prompt Tokens,Completion Tokens,Total Tokens
gpt-4o-2024-11-20,terminator,0.00216,856,2,858


Message from terminator:



Session terminated.

displaying cost...
| Agent                       | Cost                   | Prompt Tokens | Completion Tokens | Total Tokens |
|-----------------------------|------------------------|---------------|-------------------|--------------|
| executor response formatter | 0.0034300000000000003  | 1316.0        | 14.0              | 1330.0       |
| engineer response formatter | 0.0036498000000000004  | 1410.0        | 477.0             | 1887.0       |
| terminator                  | 0.00216                | 856.0         | 2.0               | 858.0        |
| control                     | 0.003745               | 1338.0        | 40.0              | 1378.0       |
| engineer                    | 0.00043979999999999996 | 1968.0        | 241.0             | 2209.0       |
| Total                       | 0.013424600000000002   | 6888.0        | 774.0             | 7662.0       |
Agent Output:
           x  np.cosh(x**2)
0  -1.000000       1.543081
1  -0.979798       1.497299
2  -0.959596       

In [62]:
results 

['PASSED',
 'PASSED',
 ['FAILED',
  "[Errno 2] No such file or directory: '/Users/milind24/Evals/cmbagent/output/data/result.csv'"],
 ['FAILED',
  "[Errno 2] No such file or directory: '/Users/milind24/Evals/cmbagent/output/data/result.csv'"],
 'PASSED',
 'PASSED',
 'PASSED',
 ['FAILED',
  "[Errno 2] No such file or directory: '/Users/milind24/Evals/cmbagent/output/data/result.csv'"],
 ['FAILED',
  "[Errno 2] No such file or directory: '/Users/milind24/Evals/cmbagent/output/data/result.csv'"],
 'PASSED',
 'PASSED',
 'PASSED',
 ['FAILED',
  "[Errno 2] No such file or directory: '/Users/milind24/Evals/cmbagent/output/data/result.csv'"],
 ['FAILED',
  "[Errno 2] No such file or directory: '/Users/milind24/Evals/cmbagent/output/data/result.csv'"],
 ['FAILED',
  "[Errno 2] No such file or directory: '/Users/milind24/Evals/cmbagent/output/data/result.csv'"],
 ['FAILED',
  "[Errno 2] No such file or directory: '/Users/milind24/Evals/cmbagent/output/data/result.csv'"],
 ['FAILED',
  "[Errno 2]

In [63]:

# Normalize into rows of the form (status, message)
structured = []
for res in results:
    if isinstance(res, str):  # PASSED
        structured.append(("PASSED", ""))
    elif isinstance(res, list) and len(res) == 2:
        structured.append((res[0], res[1]))
    else:
        structured.append(("UNKNOWN", str(res)))

# Create DataFrame
df = pd.DataFrame(structured, columns=["Status", "Message"])
print(df)

    Status                                            Message
0   PASSED                                                   
1   PASSED                                                   
2   FAILED  [Errno 2] No such file or directory: '/Users/m...
3   FAILED  [Errno 2] No such file or directory: '/Users/m...
4   PASSED                                                   
5   PASSED                                                   
6   PASSED                                                   
7   FAILED  [Errno 2] No such file or directory: '/Users/m...
8   FAILED  [Errno 2] No such file or directory: '/Users/m...
9   PASSED                                                   
10  PASSED                                                   
11  PASSED                                                   
12  FAILED  [Errno 2] No such file or directory: '/Users/m...
13  FAILED  [Errno 2] No such file or directory: '/Users/m...
14  FAILED  [Errno 2] No such file or directory: '/Users/m...
15  FAIL

In [64]:
df

,Status,Message
0,PASSED,
1,PASSED,
2,FAILED,[Errno 2] No such file or directory: '/Users/m...
3,FAILED,[Errno 2] No such file or directory: '/Users/m...
4,PASSED,
5,PASSED,
6,PASSED,
7,FAILED,[Errno 2] No such file or directory: '/Users/m...
8,FAILED,[Errno 2] No such file or directory: '/Users/m...
9,PASSED,


# Your Task


- Create 50 tasks to test calculations of all standared trigonometric functions (and/or combinations)
- it has to be in the same format as https://huggingface.co/datasets/bigcode/bigcodebench-hard and stored in csv file
- it should be all done via csv file/pandas data frame/in a single notebook.


The evaluation run should read the csv file containing all tasks and return a dataframe of result. 